In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

In [47]:
X_train=pd.read_csv('../input/month-dacon11/train_features.csv')
X_test=pd.read_csv('../input/month-dacon11/test_features.csv')
y_train=pd.read_csv('../input/month-dacon11/train_labels.csv')
submit=pd.read_csv('../input/month-dacon11/sample_submission.csv')

In [48]:
mean=X_train.iloc[:,2:].mean(axis=0)
std=X_train.iloc[:,2:].std(axis=0)

X_train_lstm=np.zeros((3125,600,6))
for uid in range(3125):
    df=X_train[X_train.id==uid].iloc[:,2:]
    X_train_lstm[uid,:,:]=(df-mean)/std
    
X_test_lstm=np.zeros((782,600,6))
for uid in range(782):
    df=X_test[X_test.id==uid+3125].iloc[:,2:]
    X_test_lstm[uid,:,:]=(df-mean)/std

In [49]:
from keras.utils import to_categorical

y=to_categorical(y_train['label'])

In [50]:
non_ex_idx=y_train[y_train['label_desc']=='Non-Exercise'].index
ex_idx=y_train[y_train['label_desc']!='Non-Exercise'].index

X_tr_ex=X_train_lstm[ex_idx,:,:]
y_tr_ex=y[ex_idx,:]

X_tr_non_ex=X_train_lstm[non_ex_idx,:,:]
y_tr_non_ex=y[non_ex_idx,:]

In [55]:
def aug(data,shift=0):
    shift_data=np.roll(data,shift,axis=1)
    return shift_data

X_train_aug=np.zeros((1607*12,600,6))
for i in range(12):
    aug_data=aug(X_tr_ex,int(np.random.random()*600))
    X_train_aug[1607*i:1607*(i+1),:,:]=aug_data
    
y_train_aug=np.vstack([y_tr_ex]*12)

In [56]:
X_tr_aug=np.vstack([X_tr_non_ex,X_train_aug])
y_tr_aug=np.vstack([y_tr_non_ex,y_train_aug])

## 1. Conv1D LSTM model

In [60]:
75*32

2400

In [68]:
model = keras.Sequential([
keras.layers.Conv1D(filters=16,kernel_size=7,strides=1,padding='same',
                    activation='elu',input_shape=(600,6)),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling1D(pool_size=2,strides=2),
keras.layers.Conv1D(filters=32,kernel_size=5,strides=1,
                                 padding='same',activation='elu'),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling1D(pool_size=2,strides=2),
keras.layers.Conv1D(filters=64,kernel_size=3,strides=1,
                    padding='same',activation='elu'),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling1D(pool_size=2,strides=2),
keras.layers.LSTM(180,return_sequences=True),
keras.layers.Flatten(),
keras.layers.Dense(2400,activation='elu'),
keras.layers.Dropout(rate=0.5),
keras.layers.Dense(200,activation='elu'),
keras.layers.Dropout(rate=0.5),
keras.layers.Dense(61,activation='softmax')
])


model.compile(loss='categorical_crossentropy',
                     optimizer='rmsprop',
                     metrics =['categorical_crossentropy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_9 (Conv1D)            (None, 600, 16)           688       
_________________________________________________________________
batch_normalization_11 (Batc (None, 600, 16)           64        
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 300, 16)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 300, 32)           2592      
_________________________________________________________________
batch_normalization_12 (Batc (None, 300, 32)           128       
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 150, 32)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 150, 64)          

In [69]:
check=keras.callbacks.ModelCheckpoint('conv1d_lstm_2_plus_elu_dropout.h5',save_best_only=True)
earlystop=keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)
history=model.fit(X_tr_aug,y_tr_aug,epochs=10,
                    validation_split=0.2,verbose=1,batch_size=32,
                    callbacks=[check,earlystop])

Epoch 1/10
521/521 [==============================] - 13s 19ms/step - loss: 4.1588 - categorical_crossentropy: 4.1588 - val_loss: 2.2788 - val_categorical_crossentropy: 2.2788
Epoch 2/10
521/521 [==============================] - 9s 18ms/step - loss: 1.9635 - categorical_crossentropy: 1.9635 - val_loss: 1.4119 - val_categorical_crossentropy: 1.4119
Epoch 3/10
521/521 [==============================] - 9s 18ms/step - loss: 1.4054 - categorical_crossentropy: 1.4054 - val_loss: 1.6323 - val_categorical_crossentropy: 1.6323
Epoch 4/10
521/521 [==============================] - 9s 18ms/step - loss: 1.1435 - categorical_crossentropy: 1.1435 - val_loss: 0.8309 - val_categorical_crossentropy: 0.8309
Epoch 5/10
521/521 [==============================] - 9s 18ms/step - loss: 0.9388 - categorical_crossentropy: 0.9388 - val_loss: 1.0185 - val_categorical_crossentropy: 1.0185
Epoch 6/10
521/521 [==============================] - 9s 18ms/step - loss: 0.8155 - categorical_crossentropy: 0.8155 - val_l

In [70]:
pred=model.predict(X_test_lstm)

In [71]:
submit.iloc[:,1:]+=pred
submit.head()

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,4.918656e-06,7.915014e-06,1.148269e-04,3.780436e-06,0.000004,5.003489e-06,0.000001,5.854615e-05,1.260445e-07,...,1.176799e-04,2.567295e-07,1.007329e-03,1.262807e-06,1.853262e-04,5.215681e-06,0.000008,9.959999e-01,5.311916e-05,2.254847e-09
1,3126,2.500643e-02,1.402023e-02,5.489491e-03,1.685648e-02,0.001704,1.059084e-03,0.002120,7.096239e-05,6.442635e-03,...,1.115812e-03,2.137121e-03,8.531470e-04,1.070393e-03,3.322269e-03,4.451168e-04,0.011152,3.651026e-03,2.792247e-04,8.414917e-02
2,3127,2.090863e-07,3.209877e-04,2.855609e-07,2.240236e-07,0.000003,5.179350e-07,0.000510,2.514527e-05,1.075335e-07,...,1.955578e-07,6.509658e-07,6.655089e-05,2.656029e-04,4.139743e-08,4.664271e-07,0.000047,9.038500e-07,5.150545e-05,9.779043e-07
3,3128,4.340739e-03,1.848456e-03,9.457666e-04,1.151156e-03,0.000099,4.265243e-04,0.000716,4.231197e-05,5.868251e-03,...,2.681555e-03,6.380713e-03,1.008750e-04,3.237496e-04,9.403006e-04,2.311459e-04,0.002425,3.366529e-04,3.385031e-05,4.255059e-02
4,3129,7.068528e-05,9.228962e-08,1.960620e-09,2.664750e-07,0.000004,3.582614e-07,0.000002,1.354550e-07,3.972700e-08,...,1.596929e-08,2.436760e-08,5.538964e-07,1.928645e-10,7.301693e-09,8.356094e-09,0.000078,1.863475e-09,8.638366e-07,3.504686e-07


In [65]:
submit.to_csv('20210220_conv1d_lstm_3.csv',index=False)

# pytorch

In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader

In [2]:
train=pd.read_csv('../input/month-dacon11/train_features.csv')
test=pd.read_csv('../input/month-dacon11/test_features.csv')
train_labels=pd.read_csv('../input/month-dacon11/train_labels.csv')
submit=pd.read_csv('../input/month-dacon11/sample_submission.csv')

In [17]:
mean=train.iloc[:,2:].mean()
std=train.iloc[:,2:].std()

X_train=np.zeros((3125,600,6))
for uid in train.id.unique():
    temp=train[train.id==uid].iloc[:,2:]
    X_train[uid,:,:]=(temp-mean)/std
    
X_test=np.zeros((782,600,6))
for uid in test.id.unique():
    temp=test[test.id==uid].iloc[:,2:]
    X_test[uid-3125,:,:]=(temp-mean)/std

In [25]:
train_labels['label'].value_counts(normalize=True)[:5]

26    0.48576
8     0.03104
28    0.01760
60    0.01536
18    0.01504
Name: label, dtype: float64

In [33]:
y_train=train_labels['label'].values

In [39]:
non_ex_idx=train_labels[train_labels['label_desc']=='Non-Exercise'].index
ex_idx=train_labels[train_labels['label_desc']!='Non-Exercise'].index

X_tr_non_ex=X_train[non_ex_idx,:,:]
y_tr_non_ex=y_train[non_ex_idx]

X_tr_ex=X_train[ex_idx,:,:]
y_tr_ex=y_train[ex_idx]

In [53]:
y_tr_ex.shape

(1607, 61)

In [44]:
def aug(data,shift=0):
    shift_data=np.roll(data,shift,axis=1)
    return shift_data

X_tr_aug=np.zeros((1607*12,600,6))
for i in range(12):
    aug_data=aug(X_tr_ex,i)
    X_tr_aug[1607*i:1607*(i+1),:,:]=aug_data

for 

array([[[ 2.21261575, -0.45138633, -1.02856296, -0.23972523,
          0.30164028,  0.44223517],
        [ 2.30731333, -0.02993881, -0.43210367,  0.01177622,
         -0.41085847, -0.46100708],
        [ 2.42108575, -0.06332133, -0.50405785,  0.026484  ,
         -0.52805207, -0.36545893],
        ...,
        [ 2.09366852, -0.10033843, -0.9930245 , -0.15697327,
          0.2758676 ,  0.31554718],
        [ 2.05575647, -0.11248974, -0.8682461 , -0.01687032,
          0.35747653,  0.25548319],
        [ 2.06399595, -0.3849974 , -0.70599755, -0.13813001,
          0.39743707,  0.27954413]],

       [[ 0.65497148,  0.00852949,  1.75114305,  1.31503641,
         -0.87010453, -0.29438948],
        [ 0.33064389,  0.14140236,  1.69088175,  0.32120978,
          0.14522759,  0.09208013],
        [ 0.26767445,  0.24392994,  1.57986201,  0.45752424,
          0.01929958,  0.09996465],
        ...,
        [ 0.68827571, -0.9999571 ,  1.46679055,  0.74893766,
          0.14833977, -0.10575588],
  